In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
    .appName('PySpark Exercises Chapter 03') \
    .getOrCreate()

spark

22/11/27 12:26:31 WARN Utils: Your hostname, karlos-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 192.168.10.22 instead (on interface enp1s0)
22/11/27 12:26:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/27 12:26:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/27 12:26:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/27 12:26:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


#### Exercise 3.3 1 By modifying the word_count_submit.py program, return the number of distinct words in Jane Austen’s Pride and Prejudice. (Hint: results contains one record for each unique word.)

In [6]:
spark.read.text('./data/gutenberg_books/1342-0.txt') \
    .select(F.split(F.col('value'), ' ').alias('line')) \
    .select(F.explode(F.col('line')).alias('word')) \
    .select(F.lower(F.col('word')).alias('word')) \
    .select(F.regexp_extract(F.col('word'), '[a-z]+', 0).alias('word')) \
    .where(F.col('word') != '') \
    .groupBy('word') \
    .count() \
    .count()

6493

#### Exercise 3.4 Taking word_count_submit.py , modify the script to return a sample of five words that appear only once in Jane Austen’s Pride and Prejudice

In [9]:
spark.read.text('./data/gutenberg_books/1342-0.txt') \
    .select(F.split(F.col('value'), ' ').alias('line')) \
    .select(F.explode(F.col('line')).alias('word')) \
    .select(F.lower(F.col('word')).alias('word')) \
    .select(F.regexp_extract(F.col('word'), '[a-z]+', 0).alias('word')) \
    .where(F.col('word') != '') \
    .groupBy('word') \
    .count() \
    .where(F.col('count') == 1) \
    .show(5)

+------------+-----+
|        word|count|
+------------+-----+
|   imitation|    1|
|     solaced|    1|
|premeditated|    1|
|     elevate|    1|
|   destitute|    1|
+------------+-----+
only showing top 5 rows



#### Exercise 3.5 1 Using the substring function (refer to PySpark’s API or the pyspark shell if needed), return the top five most popular first letters (keep only the first letter of each word).

In [32]:
spark.read.text('./data/gutenberg_books/1342-0.txt') \
    .select(F.split(F.col('value'), ' ').alias('line')) \
    .select(F.explode(F.col('line')).alias('word')) \
    .select(F.lower(F.col('word')).alias('word')) \
    .select(F.regexp_extract(F.col('word'), '[a-z]+', 0).alias('word')) \
    .where(F.col('word') != '') \
    .withColumn('first_letter', F.substring(str=F.col('word'), pos=1, len=1)) \
    .groupby(F.col("first_letter")) \
    .count() \
    .orderBy('count', ascending=False) \
    .show(5)

+------------+-----+
|first_letter|count|
+------------+-----+
|           t|16350|
|           a|13866|
|           h|10573|
|           w| 9260|
|           s| 8882|
+------------+-----+
only showing top 5 rows



#### Exercise 3.5 2 Compute the number of words starting with a consonant or a vowel. (Hint: The isin() function might be useful.)

In [37]:
vowels = ['a', 'e', 'i', 'o', 'u']

spark.read.text('./data/gutenberg_books/1342-0.txt') \
    .select(F.split(F.col('value'), ' ').alias('line')) \
    .select(F.explode(F.col('line')).alias('word')) \
    .select(F.lower(F.col('word')).alias('word')) \
    .select(F.regexp_extract(F.col('word'), '[a-z]+', 0).alias('word')) \
    .where(F.col('word') != '') \
    .withColumn('start_with_vowel', F.substring('word', 1, 1).isin(vowels)) \
    .groupBy('start_with_vowel') \
    .count() \
    .show()

+----------------+-----+
|start_with_vowel|count|
+----------------+-----+
|            true|34316|
|           false|90132|
+----------------+-----+

